
[Dataset](https://drive.google.com/file/d/1wWsrN2Ep7x6lWqOXfr4rpKGYrJhWc8z7/view)

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
import os
Root = "/content/drive/MyDrive/speech-emotion-recognition-ravdess-data.zip (Unzipped Files)/Actor_01" # Removed the file name from the path
os.chdir(Root)

In [7]:
ls

03-01-01-01-01-01-01.wav  03-01-04-01-01-01-01.wav  03-01-06-02-01-01-01.wav
03-01-01-01-01-02-01.wav  03-01-04-01-01-02-01.wav  03-01-06-02-01-02-01.wav
03-01-01-01-02-01-01.wav  03-01-04-01-02-01-01.wav  03-01-06-02-02-01-01.wav
03-01-01-01-02-02-01.wav  03-01-04-01-02-02-01.wav  03-01-06-02-02-02-01.wav
03-01-02-01-01-01-01.wav  03-01-04-02-01-01-01.wav  03-01-07-01-01-01-01.wav
03-01-02-01-01-02-01.wav  03-01-04-02-01-02-01.wav  03-01-07-01-01-02-01.wav
03-01-02-01-02-01-01.wav  03-01-04-02-02-01-01.wav  03-01-07-01-02-01-01.wav
03-01-02-01-02-02-01.wav  03-01-04-02-02-02-01.wav  03-01-07-01-02-02-01.wav
03-01-02-02-01-01-01.wav  03-01-05-01-01-01-01.wav  03-01-07-02-01-01-01.wav
03-01-02-02-01-02-01.wav  03-01-05-01-01-02-01.wav  03-01-07-02-01-02-01.wav
03-01-02-02-02-01-01.wav  03-01-05-01-02-01-01.wav  03-01-07-02-02-01-01.wav
03-01-02-02-02-02-01.wav  03-01-05-01-02-02-01.wav  03-01-07-02-02-02-01.wav
03-01-03-01-01-01-01.wav  03-01-05-02-01-01-01.wav  03-01-08-01-01-01-01.wav

In [8]:
import librosa
import soundfile
import os, glob, pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

In [9]:
#Extract features (mfcc, chroma, mel) from a sound file
def extract_feature(file_name, mfcc, chroma, mel):
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate=sound_file.samplerate
        if chroma:
            stft=np.abs(librosa.stft(X))
        result=np.array([])
        if mfcc:
            mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result=np.hstack((result, mfccs))
        if chroma:
            chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
            result=np.hstack((result, chroma))
        if mel:
            mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
            result=np.hstack((result, mel))
    return result

In [10]:
# Emotions in the RAVDESS dataset
emotions={
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fearful',
  '07':'disgust',
  '08':'surprised'
}

#Emotions to observe
observed_emotions=['calm', 'happy', 'fearful', 'disgust']

In [13]:
#Load the data and extract features for each sound file
def load_data(test_size=0.2):
    x,y=[],[]
    for file in glob.glob("/content/drive/MyDrive/speech-emotion-recognition-ravdess-data.zip (Unzipped Files)/Actor_*/*.wav"):
        file_name=os.path.basename(file)
        emotion=emotions[file_name.split("-")[2]]
        if emotion not in observed_emotions:
            continue
        feature=extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(feature)
        y.append(emotion)
    return train_test_split(np.array(x), y, test_size=test_size, random_state=9)

In [15]:
!pip install -q librosa --upgrade # ensure latest version of librosa is installed

In [17]:
#Extract features (mfcc, chroma, mel) from a sound file
def extract_feature(file_name, mfcc, chroma, mel):
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate=sound_file.samplerate
        if chroma:
            stft=np.abs(librosa.stft(X))
        result=np.array([])
        if mfcc:
            mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result=np.hstack((result, mfccs))
        if chroma:
            chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
            result=np.hstack((result, chroma))
        if mel:
            mel=np.mean(librosa.feature.melspectrogram(y=X, sr=sample_rate).T,axis=0) # Added y= to explicitly pass the signal as a keyword argument.
            result=np.hstack((result, mel))
    return result

In [18]:
#Split the dataset
x_train,x_test,y_train,y_test=load_data(test_size=0.25)

In [19]:
x_train

array([[-5.97180420e+02,  5.06116791e+01, -7.50093365e+00, ...,
         2.68607782e-05,  1.28127849e-05,  7.40735641e-06],
       [-4.81413239e+02,  4.03593674e+01, -1.13141022e+01, ...,
         1.56979309e-03,  8.16480839e-04,  3.50297720e-04],
       [-3.82501648e+02,  3.61315613e+01, -1.87694416e+01, ...,
         1.95721514e-03,  1.30252598e-03,  6.14391174e-04],
       ...,
       [-6.47284546e+02,  6.40989532e+01,  1.31187546e+00, ...,
         1.25074121e-05,  8.30124918e-06,  6.02337650e-06],
       [-6.09437500e+02,  5.49057770e+01, -1.21020401e+00, ...,
         2.64643077e-05,  1.52848388e-05,  6.05607966e-06],
       [-4.23612396e+02,  4.48980598e+01, -1.32399187e+01, ...,
         4.23118472e-04,  2.85528804e-04,  1.34936272e-04]])

In [20]:
#Get the shape of the training and testing datasets
print((x_train.shape[0], x_test.shape[0]))

(42, 14)


In [21]:
#Get the number of features extracted
print(f'Features extracted: {x_train.shape[1]}')

Features extracted: 180


In [22]:
#Initialize the Multi Layer Perceptron Classifier
model=MLPClassifier(alpha=0.01, batch_size=256, epsilon=1e-08, hidden_layer_sizes=(300,), learning_rate='adaptive', max_iter=500)

In [23]:
#Train the model
model.fit(x_train,y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:608: UserWarning: Got `batch_size` less than 1 or larger than sample size. It is going to be clipped
  warnings.warn(


MLPClassifier(alpha=0.01, batch_size=256, hidden_layer_sizes=(300,),
              learning_rate='adaptive', max_iter=500)

In [24]:
#Predict for the test set
y_pred=model.predict(x_test)

In [25]:
y_pred

array(['calm', 'calm', 'calm', 'happy', 'disgust', 'fearful', 'happy',
       'fearful', 'happy', 'disgust', 'happy', 'calm', 'calm', 'happy'],
      dtype='<U7')

In [26]:
#Calculate the accuracy of my model
accuracy=accuracy_score(y_true=y_test, y_pred=y_pred)

#Print the accuracy
print("Accuracy: {:.2f}%".format(accuracy*100))

Accuracy: 71.43%


In [27]:
from sklearn.metrics import accuracy_score, f1_score

In [28]:
f1_score(y_test, y_pred,average=None)

array([0.88888889, 1.        , 0.5       , 0.57142857])

In [29]:
import pandas as pd
df=pd.DataFrame({'Actual': y_test, 'Predicted':y_pred})
df.head(20)

,Actual,Predicted
0,calm,calm
1,calm,calm
2,calm,calm
3,fearful,happy
4,disgust,disgust
5,fearful,fearful
6,happy,happy
7,fearful,fearful
8,fearful,happy
9,disgust,disgust


In [30]:
import pickle
# Writing different model files to file
with open( 'modelForPrediction1.sav', 'wb') as f:
    pickle.dump(model,f)

In [31]:
filename = 'modelForPrediction1.sav'
loaded_model = pickle.load(open(filename, 'rb')) # loading the model file from the storage

feature=extract_feature("/content/drive/MyDrive/speech-emotion-recognition-ravdess-data.zip (Unzipped Files)/Actor_01/03-01-01-01-01-01-01.wav", mfcc=True, chroma=True, mel=True)

feature=feature.reshape(1,-1)

prediction=loaded_model.predict(feature)
prediction

array(['calm'], dtype='<U7')

In [32]:
feature

array([[-6.93497009e+02,  5.00643883e+01,  5.71450472e-01,
         1.43299656e+01,  3.33637023e+00, -2.54071975e+00,
        -4.05790901e+00, -1.07119989e+01, -7.29413891e+00,
         1.74018884e+00, -4.19064283e+00,  1.95466173e+00,
        -5.24789333e+00,  2.78142977e+00, -3.16756773e+00,
        -3.40008307e+00, -2.37803221e+00, -5.68717480e-01,
        -6.47753334e+00, -1.24320579e+00, -2.80542541e+00,
        -5.43635845e+00, -4.46875453e-01, -3.63516617e+00,
        -2.98372650e+00, -5.63902617e-01, -1.65101981e+00,
        -5.55944800e-01, -3.41018438e+00, -2.24465466e+00,
        -3.13058877e+00, -2.70089960e+00, -1.88821268e+00,
        -5.54154396e-01, -3.96459913e+00, -2.13485193e+00,
        -3.94577050e+00, -1.62457860e+00, -2.03990722e+00,
        -3.62910819e+00,  6.33734703e-01,  6.48760676e-01,
         6.21744275e-01,  6.34553194e-01,  6.60345674e-01,
         6.60464048e-01,  7.00949013e-01,  7.31556356e-01,
         7.46921480e-01,  7.23420441e-01,  7.16119647e-0